# Homework set 6

Before you turn this problem in, make sure everything runs as expected (in the menubar, select Kernel → Restart Kernel and Run All Cells...).

Please **submit this Jupyter notebook through Canvas** no later than **Mon Dec. 12, 9:00**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. A pdf version can be made using the save and export option in the Jupyter Lab file menu.**

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

Jade Dubbeld, 11692065

Maickel Hartlief, 14015277

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad
from scipy.optimize import curve_fit, minimize

In /home/makkol/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/makkol/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/makkol/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/makkol/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/makkol/anaconda3/lib/python3.7/site-packages/matplo

----------------

# Exercise 1 (4.5 points)

A bacterial population $P$ grows according to the geometric progression

$$P_t = rP_{t-1}$$

Where r is the growth rate. The following population counts $P_1 ,\ldots, P_8$ (in billions) are observed:

In [2]:
data = [0.19, 0.36, 0.69, 1.3, 2.5, 4.7, 8.5, 14]

# (a)
Read chapter 6.6 on Nonlinear Least squares. Use the Gauss-Newton Method to fit the model function $f(t, x_1, x_2) = x_1\!\cdot x_2^t$ to the data. Find estimates for the initial population $P_0=x_1$ and the growth rate $r=x_2$.

Plot the observed and modeled data and make a separate plot of the residuals.

In [3]:
def func(t, x1, x2):
    """
    Defining model function.
    
    Parameters:
    - t (nd_array): Timepoints
    - x (nd_array): x1 and x2 values
    
    Returns:
    - f (nd_array): 
    """
    
    return x1 * x2**t

t = np.arange(len(data))

residuals = []

def gn(x, y = [0.19, 0.36, 0.69, 1.3, 2.5, 4.7, 8.5, 14]):
    n = len(y)
    jacobian = np.empty((n, 2))
    summed_res = 100
    k = 0
    global residuals
    residuals = []
    while k < 1000 and summed_res > .75:
        k += 1
        y_approx = [func(t, x[0], x[1]) for t in range(n)]
        residuals.append(np.array(y_approx) - np.array(y))
        summed_res = np.linalg.norm(residuals[-1])
        for t in range(n):
            jacobian[t, 0] = -x[1] ** t
            jacobian[t, 1] = -x[0] * t * x[1] ** (t - 1)
        s = np.linalg.lstsq(jacobian, -residuals[-1], None)[0]
        x = x - s
    return y_approx, x, residuals[0]

y_approx, x, residuals = gn([], data)

print(f"Optimal values: P0 = {x[0]} and r = {x[1]}")

plt.plot(data, 'g', label = 'Original data')
plt.plot(data, 'go',label = 'Data points')
plt.plot(func(t, x[0], x[1]), 'r--', label = 'Modelled data')
plt.xlabel('Timepoints')
plt.ylabel('Data')
plt.title('Plot of original data vs modelled data')
plt.legend()
plt.show()

print(residuals)
plt.plot(residuals)
plt.xlabel('Timepoints')
plt.ylabel('Residuals')
plt.title('Plot of residuals over iterations (observed - modelled)')
plt.show()

NameError: name 'x' is not defined

# (b) 
Let $f$ be a vector valued function $f = [ f_1, \ldots, f_m ]^T$. In weighted least squares one aims to minimize the objective function
$$
  \phi(x) = \frac{1}{2} \sum_{i=1}^m W_{ii} ( f_i(x) - y_i) ^2 , \qquad
  W_{ii} = \frac{1}{\sigma_i^2} , 
$$
where $\sigma_i$ is an estimate of the standard deviation in the data point $y_i$. This is equivalent to the standard least squares problem 
$$
\min_x \frac{1}{2} \| F(x) - Y \|^2
$$
with $F(x) = T f(x)$  , $Y = T y$, where $T$ is the diagonal matrix with $\frac{1}{\sigma_i}$ on the diagonal. Assume that for each data point $y_i$ in the list above, the estimate for the standard deviation is given by
$$
  \sigma_i = 0.05 y_i .
$$ 
Perform a weighted least squares fit to obtain estimates for $P_0 = x_1$ and $r = x_2$.
Plot the observed and modeled data and make a separate plot of the residuals.

Are the results different from those of part (a)? Discuss and explain the differences.


In [ ]:
def wls(x, y = [0.19, 0.36, 0.69, 1.3, 2.5, 4.7, 8.5, 14]):
    n = len(y)
    sigmas = [.05 * x for x in y]
    I = np.eye(n)
    T = sigmas * I
    y_approx = [func(t, x[0], x[1]) for t in range(n)]
    F = T @ y_approx
    Y = T @ y
    residual = np.linalg.norm(F - Y)
    
    global residuals
    residuals.append(residual)

    return .5 * residual
    
residuals = []  
results = minimize(fun = wls,
                   x0 = [.5, 1.5],
                   args = data)    
x = results['x']

y_approx = [func(t, x[0], x[1]) for t in range(len(data))]


print(f"Optimal values: P0 = {x1} and r = {x2}")

plt.plot(data, 'g', label = 'Original data')
plt.plot(data, 'go',label = 'Data points')
plt.plot(func(t, x[0], x[1]),'r--', label = 'Modelled data')
plt.xlabel('Timepoints')
plt.ylabel('Data')
plt.title('Plot of original data vs modelled data')
plt.legend()
plt.show()

plt.plot(residuals)
plt.plot([0] * t, 'k--')
plt.xlabel('Timepoints')
plt.ylabel('Residuals')
plt.title('Plot of residuals over iterations (observed - modelled)')
plt.show()

Your answer here

----------------------
# Exercise 2 (4.5 points)



# (a) 
Let $f(x) = 1/x^2$ and $a=10^{-3}$ and $b=1$. Make a plot of the graph of $f$ on $[a,b]$. 

In [ ]:
global xVals
xVals = []

In [ ]:
# Define function f(x) in Python function
def func2(x):
    xVals.append(x)
    return 1/(x**2)

In [ ]:
# Set paramters
a = (10**(-3))
b = 1
x = np.arange(a,b,a,dtype=float)
f = func2(x)

# Plot graph of f(x) over interval [a,b]
plt.plot(x,f)
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Graph of f(x) = 1/x^2 on interval [10^(-3), 1]')
plt.show()

# (b) 
Use scipy.integrate.quad to compute $\int_a^b f(x) \, dx$ with an absolute error less than 1e-4. Compare with the true value of the integral.

In [ ]:
# Compute integral of function f(x) using scipy.integrate.quad
y, abserr = quad(func2,a,b,epsabs=1.e-4)
print(f"Calculated integral using scipy.integrate.quad: {y}")
print(f"Absolute error: {abserr}")
print("\nAbsolute error tolerance set at 1e-4")

The true value of the integral of function $f(x) = \frac{1}{x^2}$ on interval $[a,b]$ with $a = 10^{-3}$ and $b = 1$ can be calculated as follows:
\
$f(x) = \frac{1}{x^2} = x^{-2}$
\
$F(x) = -x^{-1}$
\
\
Then,
\
$\int_a^b f(x) \, dx = [F(x)]_a^b = [-x^{-1}]_{10^{-3}}^1 = - \frac{1}{1} - -\frac{1}{10^{-3}} = 999$


# (c)
scipy.integrate.quad is an adaptive integration routine (book section 8.3.6), that automatically chooses an appropriate subdivision of the interval in subintervals. Therefore in some parts of the interval $[0,1]$ the function $f$ will be evaluated more often than in other parts. Divide the interval $[0,1]$ into bins $B_i = [h * (i-1), h*i]$, with $h=0.01$. For each bin $B_i$, count the number of times $f(x)$ is evaluated with $x$ in $B_i$. Make a histogram plot of these data. In which region is there a large number of function evaluations? Can you explain this, taking into account how $f$ is defined?

Hint: To know for which $x$ the function $f$ is evaluated, you can keep a global array `xVals` in which you store all values of $x$ for which the function $f$ is called. You can use a numpy array and use numpy.append to add new values to `xVals`.


In [ ]:
a = 0
b = 1

h = 0.01

values = np.random.random_sample(1000)
values = np.sort(values)

# 100 bins for interval [0,1] with h = 0.01
bins = []
for i in range(1,101):
    bins.append((h*(i-1),h*i))

y, abserr = quad(func2,a,b)

bin_counts = [0]*len(bins)

xVals = xVals[1:-1]
print(len(xVals))

for x in xVals:
    for index,bin_borders in enumerate(bins):
        if x > bin_borders[0] and x < bin_borders[1]:
            bin_counts[index] += 1
            
plt.hist(bin_counts,range(len(bin_counts)))
plt.xlabel("Bins")
plt.ylabel("Frequency of x-values within corresponding bin")
plt.title("Distribution of function evalutions with given x-values per bin")
plt.show()

Your answer here.